# ChEMBL Data Processing

First, we obtain Cellular lines and Compound tables. And we transform them into CSV files for _pandas_ compatibility.

In [1]:
import sqlite3
import pandas as pd

database_file = 'data/chembl_27/chembl_27.db'
conn = None
try:
    conn = sqlite3.connect(database_file)
except sqlite3.Error as e:
    print(e)

cur = conn.cursor()
query = "SELECT DISTINCT c.cell_id, cell_name, cell_description, cell_source_tax_id, cell_source_organism, cellosaurus_id, " \
        "c.chembl_id FROM cell_dictionary c INNER JOIN assays a ON c.cell_id=a.cell_id"
cur.execute(query)
rows = cur.fetchall()

cell_df = pd.DataFrame(rows, columns=['cell_id', 'cell_name', 'cell_description', 'cell_source_tax_id',
                                          'cell_source_organism', 'cellosaurus_id', 'chembl_id'])
print(cell_df.head())

unable to open database file


AttributeError: 'NoneType' object has no attribute 'cursor'

In [ ]:
cell_df.to_csv(path_or_buf='data/chembl_27/cell_df.csv',header=True,index=True)

In [ ]:
cur = conn.cursor()
query = "SELECT DISTINCT ac.molregno, cs.canonical_smiles FROM activities ac " \
        "INNER JOIN assays ay ON ac.assay_id=ay.assay_id " \
        "INNER JOIN compound_structures cs ON cs.molregno=ac.molregno " \
        "INNER JOIN cell_dictionary cd ON ay.cell_id=cd.cell_id "
cur.execute(query)
rows = cur.fetchall()

comp_df = pd.DataFrame(rows, columns=['comp_id', 'smiles'])
print(comp_df.head())

In [ ]:
comp_df.to_csv(path_or_buf='data/chembl_27/comp_df.csv',header=True,index=True)

Now we select all the assays and activities of cellular lines

In [ ]:
cur = conn.cursor()
query = "SELECT ac.activity_id, ac.assay_id, ac.molregno, ay.cell_id, ac.standard_relation, ac.standard_value, " \
        "ac.standard_units, ac.standard_type, ay.assay_cell_type FROM activities ac " \
        "INNER JOIN assays ay ON ac.assay_id=ay.assay_id " \
        "WHERE ay.cell_id IS NOT NULL"
cur.execute(query)
rows = cur.fetchall()

activity_df = pd.DataFrame(rows, columns=['activity_id', 'assay_id', 'comp_id', 'cell_id',  'std_relation',
                                              'std_value', 'std_units', 'std_type', 'cell_type'])
print(activity_df.head())
activity_df.to_csv(path_or_buf='data/chembl_27/activity_df.csv',header=True,index=True)

This can be repeated for ChEMBL v25, v26, and v24

In [ ]:
import pandas as pd
import sqlite3


for database_dir in ['data/chembl_24/', 'data/chembl_25/', 'data/chembl_26/', 'data/chembl_27/']:
    database_file = database_dir + 'chembl_' + database_dir[-3:-1] + '.db'
    print(database_file)
    conn = None
    try:
        conn = sqlite3.connect(database_file)
    except sqlite3.Error as e:
        print(e)

    cur = conn.cursor()
    query = "SELECT DISTINCT c.cell_id, cell_name, cell_description, cell_source_tax_id, cell_source_organism, " \
            "cellosaurus_id, " \
            "c.chembl_id FROM cell_dictionary c INNER JOIN assays a ON c.cell_id=a.cell_id"
    cur.execute(query)
    rows = cur.fetchall()

    cell_df = pd.DataFrame(rows, columns=['cell_id', 'cell_name', 'cell_description', 'cell_source_tax_id',
                                              'cell_source_organism', 'cellosaurus_id', 'chembl_id'])
    cell_df.to_csv(path_or_buf=database_dir+'/cell_df.csv', header=True, index=True)

    cur = conn.cursor()
    query = "SELECT DISTINCT ac.molregno, md.pref_name, cs.canonical_smiles FROM activities ac " \
            "INNER JOIN molecule_dictionary md ON ac.molregno=md.molregno  " \
            "INNER JOIN assays ay ON ac.assay_id=ay.assay_id " \
            "INNER JOIN compound_structures cs ON cs.molregno=ac.molregno " \
            "WHERE ay.cell_id IS NOT NULL"
    cur.execute(query)
    rows = cur.fetchall()

    comp_df = pd.DataFrame(rows, columns=['comp_id', 'pref_name', 'smiles'])
    comp_df.to_csv(path_or_buf=database_dir+'/comp_df.csv', header=True, index=True)

    cur = conn.cursor()
    query = "SELECT ac.activity_id, ac.assay_id, ac.molregno, ay.cell_id, ac.standard_relation, ac.standard_value, " \
            "ac.standard_units, ac.standard_type, ay.assay_cell_type FROM activities ac " \
            "INNER JOIN assays ay ON ac.assay_id=ay.assay_id " \
            "INNER JOIN compound_structures cs ON cs.molregno=ac.molregno " \
            "WHERE ay.cell_id IS NOT NULL"
    cur.execute(query)
    rows = cur.fetchall()

    activity_df = pd.DataFrame(rows, columns=['activity_id', 'assay_id', 'comp_id', 'cell_id', 'std_relation',
                                                  'std_value', 'std_units', 'std_type', 'cell_type'])
    activity_df.to_csv(path_or_buf=database_dir+'/activity_df.csv', header=True, index=True)

Now, following method described in Tejera et al.
First, we select the activities reporting IC50, GI50 and CC50.

In [ ]:
import pandas as pd

activity_24 = pd.read_csv(filepath_or_buffer='data/chembl_24/activity_df.csv', index_col=0)
print(activity_24.shape)
activity_25 = pd.read_csv(filepath_or_buffer='data/chembl_25/activity_df.csv', index_col=0)
print(activity_25.shape)
activity_26 = pd.read_csv(filepath_or_buffer='data/chembl_26/activity_df.csv', index_col=0)
print(activity_26.shape)
activity_27 = pd.read_csv(filepath_or_buffer='data/chembl_27/activity_df.csv', index_col=0)
print(activity_27.shape)
activity_24 = activity_24[activity_24.std_type.isin(['IC50', 'GI50', 'CC50'])]
print(activity_24.shape, 'activities')
activity_25 = activity_25[activity_25.std_type.isin(['IC50', 'GI50', 'CC50'])]
print(activity_25.shape, 'activities')
activity_26 = activity_26[activity_26.std_type.isin(['IC50', 'GI50', 'CC50'])]
print(activity_26.shape, 'activities')
activity_27 = activity_27[activity_27.std_type.isin(['IC50', 'GI50', 'CC50'])]
print(activity_27.shape, 'activities')

activity_24.to_csv(path_or_buf='data/chembl_24/activity_24.csv', header=True, index=True)
activity_25.to_csv(path_or_buf='data/chembl_25/activity_25.csv', header=True, index=True)
activity_26.to_csv(path_or_buf='data/chembl_26/activity_26.csv', header=True, index=True)
activity_27.to_csv(path_or_buf='data/chembl_27/activity_27.csv', header=True, index=True)

We have to remove salts from chemical compounds

In [ ]:
comp_24 = pd.read_csv(filepath_or_buffer='data/chembl_24/comp_df.csv', index_col=0)
comp_list = list()
for comp_idx in range(len(comp_24)):
    smiles = comp_24.iloc[comp_idx]['smiles']
    comp_id = comp_24.iloc[comp_idx]['comp_id']
    pref_name = comp_24.iloc[comp_idx]['pref_name']
    if '.' in smiles:
        splt_smiles = str(smiles).split('.')
        smiles = max(splt_smiles, key=len)
    comp_list.append([comp_id, pref_name, smiles])
comp_24 = pd.DataFrame(comp_list, columns=['comp_id', 'pref_name', 'smiles'])
# When salts are removed, there are inconsistencies with comp_id
duplicates = comp_24[comp_24['smiles'].duplicated()]
comp_24.drop_duplicates(subset='smiles', keep='first', inplace=True, ignore_index=True)
d = pd.merge(left=duplicates, right=comp_24, how='left', left_on='smiles', right_on='smiles')
activity_24 = pd.concat([activity_24[~activity_24['comp_id'].isin(d['comp_id_x'])],
                         activity_24[~activity_24['comp_id'].map(d.set_index('comp_id_x')['comp_id_y']).isna()]])

comp_25 = pd.read_csv(filepath_or_buffer='data/chembl_25/comp_df.csv', index_col=0)
comp_list = list()
for comp_idx in range(len(comp_25)):
    smiles = comp_25.iloc[comp_idx]['smiles']
    comp_id = comp_25.iloc[comp_idx]['comp_id']
    pref_name = comp_25.iloc[comp_idx]['pref_name']
    if '.' in smiles:
        splt_smiles = str(smiles).split('.')
        smiles = max(splt_smiles, key=len)
    comp_list.append([comp_id, pref_name, smiles])
comp_25 = pd.DataFrame(comp_list, columns=['comp_id', 'pref_name', 'smiles'])
duplicates = comp_25[comp_25['smiles'].duplicated()]
comp_25.drop_duplicates(subset='smiles', keep='first', inplace=True, ignore_index=True)
d = pd.merge(left=duplicates, right=comp_25, how='left', left_on='smiles', right_on='smiles')
activity_25 = pd.concat([activity_25[~activity_25['comp_id'].isin(d['comp_id_x'])],
                         activity_25[~activity_25['comp_id'].map(d.set_index('comp_id_x')['comp_id_y']).isna()]])

comp_26 = pd.read_csv(filepath_or_buffer='data/chembl_26/comp_df.csv', index_col=0)
comp_list = list()
for comp_idx in range(len(comp_26)):
    smiles = comp_26.iloc[comp_idx]['smiles']
    comp_id = comp_26.iloc[comp_idx]['comp_id']
    pref_name = comp_26.iloc[comp_idx]['pref_name']
    if '.' in smiles:
        splt_smiles = str(smiles).split('.')
        smiles = max(splt_smiles, key=len)
    comp_list.append([comp_id, pref_name, smiles])
comp_26 = pd.DataFrame(comp_list, columns=['comp_id', 'pref_name', 'smiles'])
duplicates = comp_26[comp_26['smiles'].duplicated()]
comp_26.drop_duplicates(subset='smiles', keep='first', inplace=True, ignore_index=True)
d = pd.merge(left=duplicates, right=comp_26, how='left', left_on='smiles', right_on='smiles')
activity_26 = pd.concat([activity_26[~activity_26['comp_id'].isin(d['comp_id_x'])],
                         activity_26[~activity_26['comp_id'].map(d.set_index('comp_id_x')['comp_id_y']).isna()]])

comp_27 = pd.read_csv(filepath_or_buffer='data/chembl_27/comp_df.csv', index_col=0)
comp_list = list()
for comp_idx in range(len(comp_27)):
    smiles = comp_27.iloc[comp_idx]['smiles']
    comp_id = comp_27.iloc[comp_idx]['comp_id']
    pref_name = comp_27.iloc[comp_idx]['pref_name']
    if '.' in smiles:
        splt_smiles = str(smiles).split('.')
        smiles = max(splt_smiles, key=len)
    comp_list.append([comp_id, pref_name, smiles])
comp_27 = pd.DataFrame(comp_list, columns=['comp_id', 'pref_name', 'smiles'])
duplicates = comp_27[comp_27['smiles'].duplicated()]
comp_27.drop_duplicates(subset='smiles', keep='first', inplace=True, ignore_index=True)
d = pd.merge(left=duplicates, right=comp_27, how='left', left_on='smiles', right_on='smiles')
activity_27 = pd.concat([activity_27[~activity_27['comp_id'].isin(d['comp_id_x'])],
                         activity_27[~activity_27['comp_id'].map(d.set_index('comp_id_x')['comp_id_y']).isna()]])

comp_24.to_csv(path_or_buf='data/chembl_24/comp_24.csv', header=True, index=True)
comp_25.to_csv(path_or_buf='data/chembl_25/comp_25.csv', header=True, index=True)
comp_26.to_csv(path_or_buf='data/chembl_26/comp_26.csv', header=True, index=True)
comp_27.to_csv(path_or_buf='data/chembl_27/comp_27.csv', header=True, index=True)

activity_24.to_csv(path_or_buf='data/chembl_24/activity_24.csv', header=True, index=True)
activity_25.to_csv(path_or_buf='data/chembl_25/activity_25.csv', header=True, index=True)
activity_26.to_csv(path_or_buf='data/chembl_26/activity_26.csv', header=True, index=True)
activity_27.to_csv(path_or_buf='data/chembl_27/activity_27.csv', header=True, index=True)


Now, we classify activities into _sensitive_ and _resistant_.
An activity is _sensitive_ when concentration is lesser than or equal to 10 uM, and _resistant_ if the concentration
is bigger than 10uM.

In [ ]:
import pandas as pd
import numpy as np

activity_24 = pd.read_csv(filepath_or_buffer='data/chembl_24/activity_24.csv', index_col=0)
activity_25 = pd.read_csv(filepath_or_buffer='data/chembl_25/activity_25.csv', index_col=0)
activity_26 = pd.read_csv(filepath_or_buffer='data/chembl_26/activity_26.csv', index_col=0)
activity_27 = pd.read_csv(filepath_or_buffer='data/chembl_27/activity_27.csv', index_col=0)


def prepocess_df(df):
    # Filtering activities with Standard Units expressed in nM or uM
    df = df[df.std_units.isin(['nM', '/uM', 'umol/dm3'])]

    # Transforming uM to nM
    df.loc[df['std_units'].isin(['umol/dm3','/uM']), 'std_value'] = \
        df[df['std_units'].isin(['umol/dm3','/uM'])]['std_value'] * 1000.0
    df.loc[df['std_units'].isin(['umol/dm3','/uM']), 'std_units'] = 'nM'

    # Drop cell lines with less than 10 compounds
    activity_count = df.groupby(by=['cell_id']).size()
    df = df[df['cell_id'].isin(activity_count[activity_count >= 10].index)]

    # Column 'activity' is 1 for active, 0 for inactive
    df['activity'] = np.where(df['std_value'] <= 10000.0, 1, 0)
    return df

activity_24 = prepocess_df(activity_24)
activity_25 = prepocess_df(activity_25)
activity_26 = prepocess_df(activity_26)
activity_27 = prepocess_df(activity_27)

# Writing to disk
activity_24.to_csv(path_or_buf='data/chembl_24/activity_24.csv', header=True, index=True)
activity_25.to_csv(path_or_buf='data/chembl_25/activity_25.csv', header=True, index=True)
activity_26.to_csv(path_or_buf='data/chembl_26/activity_26.csv', header=True, index=True)
activity_27.to_csv(path_or_buf='data/chembl_27/activity_27.csv', header=True, index=True)


def summary_df(df):
    # Build summary
    df_count = df.groupby(by=['cell_id', 'comp_id', 'activity'])
    # Drop disagreements
    df_count = pd.DataFrame(df_count.size()[df_count.size()==1].index.tolist(),
                                        columns=['cell_id', 'comp_id', 'activity'])
    return df_count

summary_24 = summary_df(activity_24)
summary_25 = summary_df(activity_25)
summary_26 = summary_df(activity_26)
summary_27 = summary_df(activity_27)

summary_24[~summary_24['comp_id'].isin(comp_24['comp_id'])]

# Writing to disk
summary_24.to_csv(path_or_buf='data/chembl_24/summary_24.csv', header=True, index=True)
summary_25.to_csv(path_or_buf='data/chembl_25/summary_25.csv', header=True, index=True)
summary_26.to_csv(path_or_buf='data/chembl_26/summary_26.csv', header=True, index=True)
summary_27.to_csv(path_or_buf='data/chembl_27/summary_27.csv', header=True, index=True)
